In [1]:
#import necessary libraries
import pandas as pd
import numpy as np

C:\Users\User\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
#Reading credit dataset
credits=pd.read_csv(r"E:\ml_ai_ict\rec-data\credits.csv")

In [3]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [5]:
#Literal evaluation
from ast import literal_eval

In [6]:
credits["cast"]=credits["cast"].apply(literal_eval)
credits["crew"]=credits["crew"].apply(literal_eval)

In [7]:
#converts from list of dictionary to list
credits["cast"]=credits["cast"].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])
credits["crew"]=credits["crew"].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [8]:
credits.head()

,cast,crew,id
0,"[tom hanks, tim allen, don rickles, jim varney...","[john lasseter, joss whedon, andrew stanton, j...",862
1,"[robin williams, jonathan hyde, kirsten dunst,...","[larry j. franco, jonathan hensleigh, james ho...",8844
2,"[walter matthau, jack lemmon, ann-margret, sop...","[howard deutch, mark steven johnson, mark stev...",15602
3,"[whitney houston, angela bassett, loretta devi...","[forest whitaker, ronald bass, ronald bass, ez...",31357
4,"[steve martin, diane keaton, martin short, kim...","[alan silvestri, elliot davis, nancy meyers, n...",11862


In [12]:
#Reading keywords
keywords=pd.read_csv(r"E:\ml_ai_ict\rec-data\keywords.csv")

In [13]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [14]:
keywords.isna().sum()

id          0
keywords    0
dtype: int64

In [15]:
keywords["keywords"]=keywords["keywords"].apply(literal_eval)

In [16]:
#Converting from list of dictionary to values
keywords["keywords"]=keywords["keywords"].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])
keywords.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s=keywords.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)

s.name = 'keywords'

keywords = keywords.drop('keywords', axis=1).join(s)

#Print the head of the transformed dataframe
keywords.head()


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


,id,keywords
0,862,jealousy
0,862,toy
0,862,boy
0,862,friendship
0,862,friends


In [17]:
# Merging both data sets
merge=credits.merge(keywords,on='id')
merge.head()

,cast,crew,id,keywords
0,"[tom hanks, tim allen, don rickles, jim varney...","[john lasseter, joss whedon, andrew stanton, j...",862,jealousy
1,"[tom hanks, tim allen, don rickles, jim varney...","[john lasseter, joss whedon, andrew stanton, j...",862,toy
2,"[tom hanks, tim allen, don rickles, jim varney...","[john lasseter, joss whedon, andrew stanton, j...",862,boy
3,"[tom hanks, tim allen, don rickles, jim varney...","[john lasseter, joss whedon, andrew stanton, j...",862,friendship
4,"[tom hanks, tim allen, don rickles, jim varney...","[john lasseter, joss whedon, andrew stanton, j...",862,friends


In [24]:
#since data is huge,limitting number of records
merge = merge.iloc[:40000,:]

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

In [27]:
#Replace NaN with an empty string
merge['keywords'] = merge['keywords'].fillna("")

In [28]:
#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(merge['keywords'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(40000, 7118)

In [29]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
[(cosine_sim[1][i], merge['id'][i]) for i in cosine_sim[1].argsort()[:-20:-1]]

[(1.0, 10585),
 (1.0, 862),
 (1.0, 9279),
 (1.0, 11934),
 (1.0, 9021),
 (1.0, 11597),
 (1.0, 12232),
 (1.0, 1857),
 (1.0, 9714),
 (0.8220868024169372, 1857),
 (0.816061618936635, 2280),
 (0.816061618936635, 862),
 (0.7374829014237839, 12079),
 (0.6990237454579188, 11597),
 (0.6990237454579188, 11551),
 (0.6488481161347512, 2280),
 (0.6275305623208342, 9714),
 (0.6275305623208342, 11551),
 (0.6275305623208342, 11597)]

In [31]:
#find index from id
indices = pd.Series(merge.index, index=merge['keywords']).drop_duplicates()

In [32]:
indices

keywords
jealousy                       0
toy                            1
boy                            2
friendship                     3
friends                        4
                           ...  
lawyer                     39995
prince                     39996
water monster              39997
saving the world           39998
artificial intelligence    39999
Length: 40000, dtype: int64

In [37]:
# Function that takes in keyword as input and gives recommendations 
def content_recommender(keywords, cosine_sim=cosine_sim, merge=merge, indices=indices):

    idx = indices[keywords]

    # Get the pairwsie similarity scores of all keywords with that keyword
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the keywords based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1].all(), reverse=True)
    # Get the scores of the 10 most similar keywords. 
    sim_scores = sim_scores[1:11]

    # Get the keyword
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar keywords
    return merge['keywords'].iloc[movie_indices]

In [38]:
#Get recommendations for friendship
content_recommender('friendship')

1                   toy
2                   boy
3            friendship
4               friends
5               rivalry
6         boy next door
7               new toy
8     toy comes to life
9            board game
10        disappearance
Name: keywords, dtype: object

In [39]:
content_recommender('musical number')

1           toy
2           boy
3    friendship
4       friends
Name: keywords, dtype: object